In [1]:
from rulefit_uplift_forest import CausalRuleEnsembling
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import random
import sdv
from sdv.sampling import Condition
import torch
import pickle
from causalml.match import NearestNeighborMatch, create_table_one
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score as auc
from sklearn import preprocessing
from causalml.propensity import ElasticNetPropensityModel
from sdmetrics.reports.single_table import QualityReport
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
directory = 'temp_data/'
df = pd.read_csv(directory + 'imputed_all.csv')
df.drop(columns=['Unnamed: 0'], inplace =True)

In [3]:
obs = pd.read_csv(directory + 'matched_erich.csv')
obs.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'score', 'source', 'group'], inplace=True)

In [4]:
obs.shape

(134, 71)

In [5]:
obs.columns

Index(['TREATMENT', 'DEMO_AGE', 'OUTCOME_mRS90', 'CT1_ICHVOL', 'CT1_IVHVOL',
       'GCS_TTL', 'NIHSS_TTL', 'BP_S0', 'BP_D0', 'HIS_HYPERTENSION',
       'HIS_HYPERLIPIDEMIA', 'HIS_DM1', 'HIS_DM2', 'HIS_HF', 'HIS_AF',
       'HIS_PTCA', 'HIS_PV', 'HIS_MYOCARDIAL', 'HIS_ANTIDIABETIC',
       'HIS_ANTIHYPERTENSIVES', 'LB_WBC', 'LB_HEMOGLOBIN', 'LB_HEMATOCRIT',
       'LB_PC', 'LB_APTT', 'LB_INR', 'LB_GLUCOSE', 'LB_SODIUM', 'LB_POTASSIUM',
       'LB_CHLORIDE', 'LB_CD', 'LB_BUN', 'LB_CREATINIE', 'CT1_ICHSIDE',
       'index', 'RACE_Asian', 'RACE_Black', 'RACE_Other', 'RACE_White',
       'GENDER_Female', 'GENDER_Male', 'ETHNICITY_Hispanic',
       'ETHNICITY_Non-Hispanic', 'ICHLOC_Basal Ganglia', 'ICHLOC_Lobar',
       'ICHLOC_Other', 'ICHLOC_Thalamus', 'age', 'ich_vol', 'ivh_vol', 'gcs',
       'nihss', 'sbp', 'dbp', 'PP', 'MAP', 'pp', 'map', 'pc', 'glucose',
       'sodium', 'potassium', 'chloride', 'cd', 'bun', 'hemoglobin',
       'hematocrit', 'wbc', 'creatinie', 'aptt', 'inr'],
     

# Functions

In [6]:
def transform_var(df):
    df['age'] = (df['DEMO_AGE'] - np.mean(df['DEMO_AGE'])) / np.std(df['DEMO_AGE'])
    df['ich_vol'] = np.log(df['CT1_ICHVOL'] + 1)
    df['ivh_vol'] = np.log(df['CT1_IVHVOL'] + 1)
    df['gcs'] = (df['GCS_TTL'] - np.mean(df['GCS_TTL'])) / np.std(df['GCS_TTL'])
    df['nihss'] = np.log(df['NIHSS_TTL'] + 1)
    df['sbp'] = (df['BP_S0'] - np.mean(df['BP_S0'])) / np.std(df['BP_S0'])
    df['dbp'] = (df['BP_D0'] - np.mean(df['BP_D0'])) / np.std(df['BP_D0'])
    df['PP'] = df['BP_S0'] - df['BP_D0']
    df['MAP'] = df['BP_S0']/3 + df['BP_D0']*2/3
    df['pp'] = (df['PP'] -np.mean(df['PP'])) / np.std(df['PP'])
    df['map'] = (df['MAP'] - np.mean(df['MAP']))/np.std(df['MAP'])
    
    df['pc'] = (df['LB_PC'] - np.mean(df['LB_PC'])) / np.std(df['LB_PC'])
    df['glucose'] = (df['LB_GLUCOSE'] - np.mean(df['LB_GLUCOSE'])) / np.std(df['LB_GLUCOSE'])
    df['sodium'] = (df['LB_SODIUM'] - np.mean(df['LB_SODIUM'])) / np.std(df['LB_SODIUM'])
    df['potassium'] = (df['LB_POTASSIUM'] - np.mean(df['LB_POTASSIUM'])) / np.std(df['LB_POTASSIUM'])
    df['chloride'] = (df['LB_CHLORIDE'] - np.mean(df['LB_CHLORIDE'])) / np.std(df['LB_CHLORIDE'])
    df['cd'] = (df['LB_CD'] - np.mean(df['LB_CD'])) / np.std(df['LB_CD'])
    df['bun'] = (df['LB_BUN'] - np.mean(df['LB_BUN'])) / np.std(df['LB_BUN'])
    df['hemoglobin'] = (df['LB_HEMOGLOBIN'] - np.mean(df['LB_HEMOGLOBIN'])) / np.std(df['LB_HEMOGLOBIN'])
    df['hematocrit'] = (df['LB_HEMATOCRIT'] - np.mean(df['LB_HEMATOCRIT'])) / np.std(df['LB_HEMATOCRIT'])
    df['wbc'] = (df['LB_WBC'] - np.mean(df['LB_WBC'])) / np.std(df['LB_WBC'])
    df['creatinie'] = (df['LB_CREATINIE'] - np.mean(df['LB_CREATINIE'])) / np.std(df['LB_CREATINIE'])
    df['aptt'] = (df['LB_APTT'] - np.mean(df['LB_APTT'])) / np.std(df['LB_APTT'])
    df['inr'] = (df['LB_INR'] - np.mean(df['LB_INR'])) / np.std(df['LB_INR'])
    
    return df

In [7]:
def test_transform_var(df, train):
    # Using the training data to 
    df['age'] = (df['DEMO_AGE'] - np.mean(train['DEMO_AGE'])) / np.std(train['DEMO_AGE'])
    df['ich_vol'] = np.log(df['CT1_ICHVOL'] + 1)
    df['ivh_vol'] = np.log(df['CT1_IVHVOL'] + 1)
    df['gcs'] = (df['GCS_TTL'] - np.mean(train['GCS_TTL'])) / np.std(train['GCS_TTL'])
    df['nihss'] = np.log(df['NIHSS_TTL'] + 1)
    df['sbp'] = (df['BP_S0'] - np.mean(train['BP_S0'])) / np.std(train['BP_S0'])
    df['dbp'] = (df['BP_D0'] - np.mean(train['BP_D0'])) / np.std(train['BP_D0'])
    
    df['PP'] = df['BP_S0'] - df['BP_D0']
    df['MAP'] = df['BP_S0']/3 + df['BP_D0']*2/3
    df['pp'] = (df['PP'] -np.mean(train['PP'])) / np.std(train['PP'])
    df['map'] = (df['MAP'] - np.mean(train['MAP']))/np.std(train['MAP'])
    
    df['pc'] = (df['LB_PC'] - np.mean(train['LB_PC'])) / np.std(train['LB_PC'])
    df['glucose'] = (df['LB_GLUCOSE'] - np.mean(train['LB_GLUCOSE'])) / np.std(train['LB_GLUCOSE'])
    df['sodium'] = (df['LB_SODIUM'] - np.mean(train['LB_SODIUM'])) / np.std(train['LB_SODIUM'])
    df['potassium'] = (df['LB_POTASSIUM'] - np.mean(train['LB_POTASSIUM'])) / np.std(train['LB_POTASSIUM'])
    df['chloride'] = (df['LB_CHLORIDE'] - np.mean(train['LB_CHLORIDE'])) / np.std(train['LB_CHLORIDE'])
    df['cd'] = (df['LB_CD'] - np.mean(train['LB_CD'])) / np.std(train['LB_CD'])
    df['bun'] = (df['LB_BUN'] - np.mean(train['LB_BUN'])) / np.std(train['LB_BUN'])
    df['hemoglobin'] = (df['LB_HEMOGLOBIN'] - np.mean(train['LB_HEMOGLOBIN'])) / np.std(train['LB_HEMOGLOBIN'])
    df['hematocrit'] = (df['LB_HEMATOCRIT'] - np.mean(train['LB_HEMATOCRIT'])) / np.std(train['LB_HEMATOCRIT'])
    df['wbc'] = (df['LB_WBC'] - np.mean(train['LB_WBC'])) / np.std(train['LB_WBC'])
    df['creatinie'] = (df['LB_CREATINIE'] - np.mean(train['LB_CREATINIE'])) / np.std(train['LB_CREATINIE'])
    df['aptt'] = (df['LB_APTT'] - np.mean(train['LB_APTT'])) / np.std(train['LB_APTT'])
    df['inr'] = (df['LB_INR'] - np.mean(train['LB_INR'])) / np.std(train['LB_INR'])
    
    return df

In [8]:
pre_treatment_var = ['age', 'ich_vol', 'ivh_vol', 'gcs', 'nihss', 'sbp', 'dbp', 'pp', 'map', 
                     'RACE_Asian', 'RACE_Black', 'RACE_Other', 'RACE_White', 'GENDER_Male', 'GENDER_Female',
                     'ETHNICITY_Hispanic','ETHNICITY_Non-Hispanic',
                     
                     'HIS_HYPERTENSION', 'HIS_HYPERLIPIDEMIA', 'HIS_DM2',
                     'HIS_DM1', 'HIS_HF', 'HIS_AF', 'HIS_PTCA', 'HIS_PV','HIS_MYOCARDIAL', 
                     'HIS_ANTIDIABETIC', 'HIS_ANTIHYPERTENSIVES',
                     
                     'CT1_ICHSIDE','ICHLOC_Basal Ganglia', 'ICHLOC_Lobar', 'ICHLOC_Thalamus', 'ICHLOC_Other',
                     
                     'wbc', 'hemoglobin','hematocrit', 'pc', 'aptt', 'inr', 'glucose',
                     'sodium', 'potassium', 'chloride', 'cd', 'bun','creatinie']

In [9]:
from sdv.tabular import TVAE
trial_temp = df[(df['source'] == 'atach2')&(df['group']=='train')].copy()
trial_temp.reset_index(inplace=True, drop=True)
erich_temp = df[df['source'] == 'erich'].copy()
df_train_syn = pd.concat([trial_temp, erich_temp], axis=0)
len(df_train_syn)

3506

In [10]:
df_train_syn.drop(columns=['group', 'index'], inplace=True)

In [12]:
# SEED_VALUE = 10
# np.random.seed(SEED_VALUE)
# torch.manual_seed(SEED_VALUE)
# tvae_model = TVAE(cuda=True)
# tvae_model.fit(df_train_syn)

In [13]:
#tvae_model.save('temp_data/TVAE_comb_v1.pkl')

In [11]:
tvae_model = TVAE.load('temp_data/TVAE_comb_v1.pkl')

In [12]:
SEED_VALUE = 20
np.random.seed(SEED_VALUE)
torch.manual_seed(SEED_VALUE)
cond1 = Condition({'TREATMENT':1}, 500)
cond2 = Condition({'TREATMENT':0}, 500)

new_data_t2 = tvae_model.sample_conditions(conditions=[cond1])
new_data_c2 = tvae_model.sample_conditions(conditions=[cond2])

Sampling conditions: 100%|██████████| 500/500 [00:00<00:00, 941.04it/s] 


In [13]:
syn2 = pd.concat([new_data_t2, new_data_c2])
syn2.reset_index(drop=True, inplace=True)
syn2.to_csv('temp_data/tvae_syn_new.csv')
syn2.drop(columns=['source'], inplace=True)
syn2 = pd.get_dummies(syn2)
syn2['source'] = ['syn'] * len(syn2)
syn2['group'] = ['train']*len(syn2)

In [14]:
df_new = df.copy()
df_new.drop(columns=['source', 'group'], inplace=True)
df_new = pd.get_dummies(df_new)
df_new['source'] = df['source']
df_new['group'] = df['group']

In [15]:
set(df_new.columns) - set(syn2.columns)

{'ICHLOC_Other', 'index'}

In [16]:
syn_cols = list(syn2.columns)
all_cols = list(df_new.columns)
for f in all_cols:
    if f not in syn_cols:
        syn2[f] = [0] * len(syn2)

In [17]:
merge1 = pd.concat([df_new, syn2])

In [18]:
trial_train = merge1[(merge1['source'] == 'atach2')&(merge1['group']=='train')].copy()
trial_train.reset_index(inplace=True, drop=True)
df_test = merge1[merge1['group'] == 'test'].copy()
df_train = merge1[merge1['group'] == 'train'].copy()
syn = merge1[merge1['source'] == 'syn'].copy()

In [19]:
matched_ids = list(obs['index'])
matched_obs = merge1[merge1['index'].isin(matched_ids)]
matched_obs.shape

(134, 49)

In [20]:
merge_temp1 = pd.concat([trial_train, matched_obs, syn])
merge_temp1 = transform_var(merge_temp1)
X = np.array(merge_temp1[pre_treatment_var])
y = np.array(merge_temp1['TREATMENT'])
pm_lgr = LogisticRegression(penalty='none', max_iter=3000)
pm_lgr.fit(X, y)
clip_bounds = (1e-3, 1-1e-3)
score_lgr1 = np.clip(pm_lgr.predict_proba(X)[:, 1], *clip_bounds)
print('AUC score: {:.6f}'.format(auc(y, score_lgr1)))

AUC score: 0.855437


In [21]:
merge_temp1['score'] = score_lgr1
t_syn2 = merge_temp1[(merge_temp1['TREATMENT'] == 1) & (merge_temp1['source'] == 'syn')].copy()
c_syn2 = merge_temp1[(merge_temp1['TREATMENT'] == 0) & (merge_temp1['source'] == 'syn')].copy()
t_real = merge_temp1[(merge_temp1['TREATMENT'] == 1) & (merge_temp1['source'] != 'syn')].copy()
c_real = merge_temp1[(merge_temp1['TREATMENT'] == 0) & (merge_temp1['source'] != 'syn')].copy()
print(t_syn2.shape), print(c_syn2.shape), print(t_real.shape), print(c_real.shape)

temp21 = pd.concat([t_syn2, c_real])
temp21['treatment'] = 1 - temp21['TREATMENT']
temp21.index = np.arange(0, len(temp21))
temp22 = pd.concat([c_syn2, t_real])
temp22.index = np.arange(0, len(temp22))

# PSM21: use real control to find matched treatment
psm21 = NearestNeighborMatch(caliper=0.2, replace=False, ratio=1, random_state=2205)
matched_syn21 = psm21.match(data=temp21, treatment_col='treatment',score_cols=['score'])

# PSM22: use real treatment to find matched control
psm22 = NearestNeighborMatch(caliper=0.2, replace=False, ratio=1, random_state=2205)
matched_syn22 = psm22.match(data=temp22, treatment_col='TREATMENT',score_cols=['score'])

(500, 74)
(500, 74)
(397, 74)
(537, 74)


In [22]:
matched_syn1 = pd.concat([matched_syn21[matched_syn21['TREATMENT'] == 1], matched_syn22[matched_syn22['TREATMENT'] == 0]])
len(matched_syn1)

262

In [23]:
len(matched_syn21[matched_syn21['TREATMENT'] == 1]), len(matched_syn22[matched_syn22['TREATMENT'] == 0])

(100, 162)

In [24]:
merge2 = pd.concat([trial_train, matched_obs, matched_syn1])
merge2 = transform_var(merge2)
y = np.array(merge2['TREATMENT'])
X = merge2[pre_treatment_var].values
pm_lgr = LogisticRegression(penalty='none', max_iter=3000)
pm_lgr.fit(X, y)
clip_bounds = (1e-3, 1-1e-3)
score_lgr = np.clip(pm_lgr.predict_proba(X)[:, 1], *clip_bounds)
print('AUC score: {:.6f}'.format(auc(y, score_lgr)))

AUC score: 0.680308


In [25]:
len(merge2), len(merge2[merge2['TREATMENT'] == 1]), len(merge2[merge2['TREATMENT']==0])

(1196, 497, 699)

In [26]:
eval_merge2 = create_table_one(merge2, 'TREATMENT', pre_treatment_var)
a = 0
b = 0
# Calculate average smd on ERICH + ATACH2
for i in range(len(eval_merge2['SMD'])):
    if (eval_merge2['SMD'][i] !='') & ~(pd.isna(eval_merge2['SMD'][i])):
        a += abs(eval_merge2['SMD'][i])
        b += 1
a/b

0.07918913043478261

In [27]:
eval_merge2 = create_table_one(merge_temp1, 'TREATMENT', pre_treatment_var)
a = 0
b = 0
# Calculate average smd on ERICH + ATACH2
for i in range(len(eval_merge2['SMD'])):
    if (eval_merge2['SMD'][i] !='') & ~(pd.isna(eval_merge2['SMD'][i])):
        a += abs(eval_merge2['SMD'][i])
        b += 1
a/b

0.27833043478260877

Hyperprameter setting:
- nreg = 5
- max_sample_tree = 80
- max_treatment_tree = 30

for tree:
- nreg = 5
- max_sample_tree = 80
- max_treatment_tree = 30

# Experiments after matching

In [28]:
test = df_test.copy()
test = test_transform_var(test, merge2)
train_X = np.array(merge2[pre_treatment_var])
test_X = np.array(test[pre_treatment_var])
train_treatment=(merge2['TREATMENT']!=0).astype(int).values
test_treatment=(test['TREATMENT']!=0).astype(int).values
y_train = (merge2['OUTCOME_mRS90']).values
y_test = (test['OUTCOME_mRS90']).values
y_bin_train = np.array([0] * len(y_train))
y_bin_train[np.where(y_train <= 2)] = 1
y_bin_test = np.array([0] * len(y_test))
y_bin_test[np.where(y_test <= 2)] = 1
treatment_train = ['control'] * len(train_treatment)
for i in range(len(train_treatment)):
    if train_treatment[i] == 1:
        treatment_train[i] = 'treatment'
treatment_train = np.array(treatment_train)

In [29]:
train_X.shape, test_X.shape

((1196, 46), (200, 46))

In [30]:
from uplift_forest_customed import UpliftTreeNew
model_tree = UpliftTreeNew(n_reg=5, min_samples_leaf = 80, min_samples_treatment = 30, random_state=100, control_name='control')
model_tree.fit(train_X, treatment_train, y_bin_train)
model_tree.eval_qini(test_X, y_bin_test, test_treatment)

-0.0009039822604434409

In [31]:
ctgan_tree_rules = model_tree.get_rules()
len(ctgan_tree_rules)

3

In [32]:
ctgan_tree_rules

[1, 1, 1]

In [33]:
qini_max2 = -1
res2 = {'res_tree':[], 'res_lasso':[]}
for seed in range(0, 30):
    
    model_temp = CausalRuleEnsembling(
         tree_depth = 3, 
         tree_eval_func = 'KL', 
         n_reg=5, 
         n_estimator = 100,   
         min_samples_leaf = 80, 
         min_samples_treatment = 30, 
         model_type='rl', 
         lin_standardise=False,
         random_state = seed)
    model_temp.fit(train_X, treatment_train, y_bin_train, pre_treatment_var)
    a, b = model_temp.eval_qini(test_X, y_bin_test, test_treatment)
    
    res2['res_tree'].append(a)
    res2['res_lasso'].append(b)
    print('Seed round:', seed, b)
    if b > qini_max2:
        qini_max2 = b
        final_model2 = model_temp

Seed round: 0 0.07056902069799384
Seed round: 1 0.034126669602770686
Seed round: 2 0.07371328170835421
Seed round: 3 0.02584400520588578
Seed round: 4 0.05745719106742597
Seed round: 5 0.05522481487927093
Seed round: 6 0.046737384329793456
Seed round: 7 0.0632918704842996
Seed round: 8 0.05354139503481883
Seed round: 9 0.07033968106226106
Seed round: 10 0.0973596406928072
Seed round: 11 0.032363778033775194


Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.629e-03, tolerance: 1.091e-03


Seed round: 12 0.06404794937330954
Seed round: 13 0.09518161317977629
Seed round: 14 0.1090626597555841
Seed round: 15 0.03388444098526454
Seed round: 16 0.05247523543285911
Seed round: 17 0.05003562691571137
Seed round: 18 0.11923502191566737
Seed round: 19 0.05044180515043068
Seed round: 20 0.11675782640684733


Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.094e-03, tolerance: 8.420e-04


Seed round: 21 0.05752363114019471
Seed round: 22 0.07983957392055462
Seed round: 23 0.02810610937542351


Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.136e-04, tolerance: 7.375e-04


Seed round: 24 0.03246314791153111
Seed round: 25 0.03818064151828177
Seed round: 26 0.04913325931096152
Seed round: 27 0.05651880908074845
Seed round: 28 0.07629832635331084
Seed round: 29 0.05184367020012397


In [36]:
with open('results/tvae_tree_mrs2.txt', 'w') as filehandle:
    for listitem in res2['res_tree']:
        filehandle.write(f'{listitem}\n')
        
with open('results/tvae_ensemble_mrs2.txt', 'w') as filehandle:
    for listitem in res2['res_lasso']:
        filehandle.write(f'{listitem}\n')

In [34]:
np.mean(res2['res_tree']),np.mean(res2['res_lasso'])

(0.060718395588974135, 0.06138660269086791)

In [35]:
np.std(res2['res_tree']),np.std(res2['res_lasso'])

(0.02537838086067204, 0.02519045091247927)

In [36]:
rules = final_model2.get_rules()
rules = rules[(rules['type']=='rule')]
rules_ = rules[(rules['coef'] != 0)]
print(len(rules_))
print(len(rules))

168
209


In [40]:
final_model2.eval_qini(test_X, y_bin_test, test_treatment)

(0.11485044554004102, 0.11923502191566737)

In [41]:
filename = 'final_data/final_tave_model_mrs2_500.sav'
pickle.dump(final_model2, open(filename, 'wb'))

In [42]:
sum(merge2['TREATMENT']), len(merge2) - sum(merge2['TREATMENT']), len(merge2)

(497, 699, 1196)